In [1]:
import os

local_DataDir_name='SimulatorData1'
local_csv_file_name='driving_log.csv'
print(os.getcwd())

/Users/mparuchuri/Documents/Madhu's Studies/SelfDrivingCar/Projects/behaviourCloning


In [2]:
uplevelDir = os.path.dirname(os.getcwd())
print(os.path.dirname(os.getcwd()))

/Users/mparuchuri/Documents/Madhu's Studies/SelfDrivingCar/Projects


In [3]:
print(os.listdir(uplevelDir))

['.DS_Store', '.ipynb_checkpoints', 'behaviourCloning', 'DeepLearning', 'GitProjects', 'MiniFlow', 'notMNIST.pickle', 'SimulatorData', 'SimulatorData1', 'Untitled.ipynb']


In [4]:
print(os.listdir(os.path.join(uplevelDir,local_DataDir_name)))
data_dir = os.path.join(uplevelDir,local_DataDir_name)

['.DS_Store', 'driving_log.csv', 'IMG']


In [5]:
csv_file_name = os.path.join(data_dir,local_csv_file_name)
print(csv_file_name)

/Users/mparuchuri/Documents/Madhu's Studies/SelfDrivingCar/Projects/SimulatorData1/driving_log.csv


In [6]:
import csv
image_metadata = []
with open(csv_file_name,'r') as csvfile:
    data = csv.reader(csvfile)
    for line in data:
        image_metadata.append(line)



In [7]:
print(len(image_metadata))

2068


In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [9]:
#Split data into train and valid

image_metadata = shuffle(image_metadata)
train_samples,validation_samples = train_test_split(image_metadata,test_size=0.2,random_state=2)

print("Train_samples: ",len(train_samples))
print("Valid_samples: ",len(validation_samples))

Train_samples:  1654
Valid_samples:  414


In [10]:
# each generator call
# read batch of data
#Batch_Size=50
def getBatchData(image_metadata,Batch_Size,fetch_side_camera_data=False):
    numSamples = len(image_metadata)
    while True:
        image_metadata = shuffle(image_metadata)
        for offset in range(0,numSamples,Batch_Size):
            batch_meta = image_metadata[offset : offset+Batch_Size]
            image_data=[]
            image_labels=[]
            for line in batch_meta:
                angle = float(line[3])
                correction_factor = 0.1
                if fetch_side_camera_data and angle != 0.0:
                #if fetch_side_camera_data:
                    image_index = [0,1,2]
                    angle_index = [angle,angle+correction_factor,angle-correction_factor]
                else:
                    image_index = [0]
                    angle_index = [angle]
                for index,steer_angle in zip(image_index,angle_index):
                    image = os.path.join(data_dir,os.path.join(line[index].split(os.path.sep)[-2],line[index].split(os.path.sep)[-1]))
                    image_array = plt.imread(image)
                    image_data.append(image_array)
                    image_labels.append(steer_angle)
                    
                    flipped_image = cv2.flip(image_array,1)
                    image_data.append(flipped_image)
                    #image_labels.append(-steer_angle if steer_angle != 0.0 else steer_angle)
                    image_labels.append(steer_angle * -1.0)
            image_data = np.array(image_data)
            image_labels = np.array(image_labels)
            yield shuffle(image_data,image_labels)
            

In [11]:
batch_size = 50
train_generator = getBatchData(train_samples,Batch_Size=batch_size,fetch_side_camera_data=True)
validation_generator = getBatchData(validation_samples,Batch_Size=batch_size,fetch_side_camera_data=False)

In [12]:
#Keras Model

from keras.layers import Input,Flatten,Dense,Dropout,Activation,Lambda
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D
from keras.initializers import RandomNormal, Constant
import tensorflow as tf
tf.python.control_flow_ops = tf

Using TensorFlow backend.


In [13]:
initialize_weights = RandomNormal(mean=0.0, stddev=0.05, seed=None)
initialize_bias = Constant(value=0.1)

model = Sequential()
model.add(Cropping2D(cropping=((80,20),(0,0)),input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.))

model.add(Conv2D(32,(11,11),strides=(1, 1),padding='same',
                 activation='relu',use_bias=True,kernel_initializer=initialize_weights,
                 bias_initializer=initialize_bias))

model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64,(11,11),strides=(1, 1),padding='same',
                 activation='relu',use_bias=True,kernel_initializer=initialize_weights,
                 bias_initializer=initialize_bias))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(1))

In [14]:
model.compile(optimizer='adam',loss='mse')
model.fit_generator(train_generator, validation_data=validation_generator,verbose=1, validation_steps=math.ceil(len(validation_samples) / batch_size), epochs=3,steps_per_epoch = math.ceil(len(train_samples) / batch_size))

Epoch 1/3
34/34 [==============================] - 1340s - loss: 157.0443 - val_loss: 0.0135

In [15]:
model.save('modelwithconv1Deeper.h5')